In [2]:
import pandas as pd
import numpy as np

In [1]:
import pandas as pd
import warnings 
# import geopandas as gpd
import numpy as np
# from shapely.geometry import Polygon, LineString, Point
warnings.filterwarnings(action='ignore')

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
from sklearn.preprocessing import StandardScaler

plt.rcParams['font.family'] = 'Malgun Gothic'

# 경감지수 전처리

## 학교정보 및 상담 데이터

In [3]:
school_info = pd.read_csv('./Data/서울시고등학교_기본정보.csv', encoding='cp949')
councel = pd.read_csv('./Data/서울시고등학교_교육여건_상담.csv', encoding='cp949')
# 학교 기본정보 + 교육여건_상담 데이터 병합
school_info = school_info[['학교코드','위도','경도','법정동코드']]
school = pd.merge(councel, school_info, on='학교코드', how='left')
school.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   구                       320 non-null    object 
 1   학교명                     320 non-null    object 
 2   학교코드                    320 non-null    object 
 3   내부상담_실시여부               320 non-null    int64  
 4   외부상담_실시여부               320 non-null    int64  
 5   wee클래스_운영여부             320 non-null    int64  
 6   학교폭력_예방교육_평균교육시간(1학급당)  320 non-null    float64
 7   위도                      320 non-null    float64
 8   경도                      320 non-null    float64
 9   법정동코드                   320 non-null    int64  
dtypes: float64(3), int64(4), object(3)
memory usage: 27.5+ KB


## 학교 500m 내 경찰관서수

In [6]:
police_loc = pd.read_csv('./Data/경찰청_경찰관서 위치.csv', encoding='cp949')
Police_station = police_loc.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
Police_station

,청,서,지구대파출소,경도,위도
0,서울청,서울강남경찰서,논현1파출소,127.029316,37.513791
1,서울청,서울강남경찰서,논현2파출소,127.033875,37.515220
2,서울청,서울강남경찰서,삼성1파출소,127.060440,37.514852
3,서울청,서울강남경찰서,삼성2파출소,127.048063,37.511289
4,서울청,서울강남경찰서,서울강남경찰서,127.067177,37.509036
...,...,...,...,...,...
269,서울청,서울혜화경찰서,종로5가파출소,127.001606,37.570727
270,서울청,서울혜화경찰서,창신파출소,127.013702,37.578482
271,서울청,서울혜화경찰서,혜화파출소,127.001422,37.586135
272,서울청,서울혜화경찰서,효제파출소,127.003606,37.574259


In [7]:
# !pip install haversine

In [8]:
from tqdm.notebook import tqdm
from tqdm import tqdm, tqdm_notebook
from haversine import haversine

In [9]:
# 경찰관서수 구하는 함수 
def close_ps(school, Police_station, column) : 
    for i in tqdm(range(len(school))) : 
        distance = [] 
        for j in range(len(Police_station)) : 
            schoolloc = tuple(school[['경도','위도']].iloc[i])
            Police_stationloc = tuple(Police_station[['경도','위도']].iloc[j])
            distance.append(haversine(schoolloc, Police_stationloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 500] 
        school[column][i] = len(distance)
        
    return school

In [10]:
school["500미터내_파출소개수"] = 0
school = close_ps(school, Police_station, "500미터내_파출소개수")

100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [01:43<00:00,  3.09it/s]


## 학교 500m 이내에 위치한 CCTV 대수

In [12]:
CCTV = pd.read_csv('./Data/서울시CCTV_방범.csv',encoding='cp949', thousands = ',').iloc[:,:-1]
CCTV

,자치구,CCTV 용도,위도,경도
0,중랑구,방범,37.580608,127.083908
1,중랑구,방범,37.588905,127.097982
2,중랑구,방범,37.581651,127.089158
3,중랑구,방범,37.599900,127.090018
4,중랑구,방범,37.593811,127.092786
...,...,...,...,...
29513,강남구,방범,37.519440,127.038720
29514,강남구,방범,37.530720,127.031450
29515,강남구,방범,37.494170,127.063010
29516,강남구,방범,37.530148,127.023759


In [13]:
# cctv대수 구하는 함수 
def close_CCTV(school, CCTV, column) : 
    for i in tqdm(range(len(school))) : 
        distance = [] 
        for j in range(len(CCTV)) : 
            schoolloc = tuple(school[['경도','위도']].iloc[i])
            CCTVloc = tuple(CCTV[['경도','위도']].iloc[j])
            distance.append(haversine(schoolloc, CCTVloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 500] 
        school[column][i] = len(distance)
        
    return school

In [14]:
school["500미터내_CCTV개수"] = 0
school = close_CCTV(school, CCTV, "500미터내_CCTV개수")

  2%|█▎                                                                              | 5/320 [03:10<3:20:25, 38.18s/it]

KeyboardInterrupt: 

In [ ]:
school.to_csv('./Data/경감지표_전처리.csv', index=False)